# Redis tutorial

## Prerequisites

### Documentation

You will find all documentation for :
* [Redis commands](https://redis.io/commands)
* [Redis python client](https://redis-py.readthedocs.io/en/latest/)

Though the tutorial asks you to use Redis commands, they all have an equivalent in the Python client library.

### Import libraries

In [1]:
import redis

In [2]:
r = redis.Redis(host='localhost', port=6379)

In [3]:
r.set('foo', 'bar')
value = r.get('foo')
print(value)

b'bar'


You can launch a terminal aside, connect to your server with a Redis client and check that the value is still present :

```bash
vagrant@nosql:~$ redis-cli
127.0.0.1:6379> get foo
"bar"
```

## I. Quick start

### Strings and numbers

**Q** : Store the string `hello world` in key `greetings` with a `SET` command. Then use a `GET` command to retrieve it.

In [4]:
# Stocker la chaîne "hello world" dans la clé "greetings"
r.set('greetings', 'hello world')
# Récupérer la valeur de la clé "greetings"
value = r.get('greetings')
print(value)

b'hello world'


**Q** : Create a `connections` key with value 0. Find the `INCR` command to increment its value, then display it. 

Try to increment the value for `greetings`.

In [5]:
# Étape 1 : Créer une clé "connections" avec la valeur 0
r.set('connections', 0)
print(r.get('connections'))

# Étape 2 : Incrémenter la valeur de "connections"
r.incr('connections')
print(r.get('connections'))


b'0'
b'1'


### Lists

**Q** : Create a `colors` list with strings `red`, `black` and `blue`, using the `RPUSH` command.

In [6]:
# Créer une liste "colors" et ajouter des éléments avec RPUSH
r.rpush('colors', 'red', 'black', 'blue')

3

**Q** : Display the second element of the list with `LINDEX`

In [7]:
print(r.lindex('colors', 1))

b'black'


**Q** : Display the full list with `LRANGE`

In [8]:
print(r.lrange('colors', 0, -1))

[b'red', b'black', b'blue']


### Using sets

**Q** : Create a `nosql` set containing values `redis`, `mongodb` and `orientdb`, using the `SADD` command.

In [9]:
r.sadd('nosql', "redis", 'mongodb', 'orientdb')

3

**Q** : Test if `mysql` is inside the set, using `SISMEMBER`

In [10]:
r.sismember('nosql', 'redis')

1

**Q** : Add `hbase` to the set with `SADD`, then display all elements of the set with `SMEMBERS`. Try to add `redis` again and see what happens.

In [11]:
# Add 'hbase' to the 'nosql' set
r.sadd('nosql', 'redis')
print(r.sismember('nosql', 'redis'))

1


In [12]:
# Retrieve all the keys in the database
all_keys = r.keys('*')
print(all_keys)
print(r.get(all_keys[0]))

[b'foo', b'greetings', b'nosql', b'colors', b'connections']
b'bar'


### Using sorted sets

Sorted Sets are similar to Redis Sets with the unique feature of values stored in a set. The difference is, every member of a Sorted Set is associated with a score, that is used in order to take the sorted set ordered, from the smallest to the greatest score.

**Q** : Using `ZADD`, create a sorted set `top14` with the following entries :

```
score city
10 Agen 
33 Bordeaux 
32 Brive 
29 Castres 
38 Clermont 
24 Grenoble 
26 La Rochelle 
32 Montpellier 
14 Oyonnax 
20 Pau 
40 Racing 
22 Stade Français 
36 Toulon 
36 Toulouse
```

In [13]:
# Create a sorted set 'top14' with the provided scores and cities
r.zadd('top14', {
    'Agen': 10,
    'Bordeaux': 33,
    'Brive': 32,
    'Castres': 29,
    'Clermont': 38,
    'Grenoble': 24,
    'La Rochelle': 26,
    'Montpellier': 32,
    'Oyonnax': 14,
    'Pau': 20,
    'Racing': 40,
    'Stade Français': 22,
    'Toulon': 36,
    'Toulouse': 36
})
sorted_cities = r.zrange('top14', 0, -1, withscores=True)
print(sorted_cities)


[(b'Agen', 10.0), (b'Oyonnax', 14.0), (b'Pau', 20.0), (b'Stade Fran\xc3\xa7ais', 22.0), (b'Grenoble', 24.0), (b'La Rochelle', 26.0), (b'Castres', 29.0), (b'Brive', 32.0), (b'Montpellier', 32.0), (b'Bordeaux', 33.0), (b'Toulon', 36.0), (b'Toulouse', 36.0), (b'Clermont', 38.0), (b'Racing', 40.0)]


**Q** : Fetch the score for `Toulon` with `ZSCORE`, and its ranking with `ZRANK`.

In [14]:
# Fetch the score for 'Toulon'
toulon_score = r.zscore('top14', 'Toulon')
print(f"Score for Toulon: {toulon_score}")

# Fetch the ranking for 'Toulon' (0 is the highest rank)
toulon_ranking = r.zrank('top14', 'Toulon')
print(f"Ranking for Toulon: {toulon_ranking}")


Score for Toulon: 36.0
Ranking for Toulon: 10


**Q** : `ZRANK` starts at 0 and scores are sorted from lowest to highest, so we should use the `ZREVRANK` for a true ranking of our cities.

In [15]:
# Fetch the score for 'Toulon'
toulon_score = r.zscore('top14', 'Toulon')
print(f"Score for Toulon: {toulon_score}")

# Fetch the true ranking for 'Toulon' (0 is the highest rank)
toulon_true_ranking = r.zrevrank('top14', 'Toulon')
print(f"True Ranking for Toulon: {toulon_true_ranking}")


Score for Toulon: 36.0
True Ranking for Toulon: 3


**Q** : Find the commands to display :
* the 3 best teams
* teams with more than 35 points

In [16]:
# Retrieve the top 3 teams with the highest scores
top_3_teams = r.zrevrange('top14', 0, 2, withscores=True)
print("Top 3 teams:", top_3_teams)
# Retrieve teams with scores greater than 35
teams_above_35 = r.zrangebyscore('top14', 35, '+inf', withscores=True)
print("Teams with more than 35 points:", teams_above_35)

Top 3 teams: [(b'Racing', 40.0), (b'Clermont', 38.0), (b'Toulouse', 36.0)]
Teams with more than 35 points: [(b'Toulon', 36.0), (b'Toulouse', 36.0), (b'Clermont', 38.0), (b'Racing', 40.0)]


### Using dictionaries

**Q** : Create a dictionary `user:1` with `HMSET` with properties `id (1), name (Jean), age (22)`. Display it with `HGETALL`

In [17]:
# Create a dictionary 'user:1' with the properties using HSET
r.hset('user:1', mapping={
    'id': 1,
    'name': 'Jean',
    'age': 22
})

# Display the dictionary with HGETALL
user_info = r.hgetall('user:1')
print(user_info)


{b'id': b'1', b'name': b'Jean', b'age': b'22'}


**Q** : Add a `city (Lyon)` property and rename the user from `Jean` to `Paul`.

In [18]:
# Add a 'city' property to the 'user:1' hash
r.hset('user:1', 'city', 'Lyon')
# Rename the user from 'Jean' to 'Paul' by updating the 'name' field
r.hset('user:1', 'name', 'Paul')
# Display the updated 'user:1' dictionary
user_info = r.hgetall('user:1')
print(user_info)

{b'id': b'1', b'name': b'Paul', b'age': b'22', b'city': b'Lyon'}


## Modelling a query cache with Redis

You are modeling data from a REST query cache system with Redis.
A request is identified by the http method and its url (without the protocol).
The content of the request is stored as it is to be returned on demand.

**Q** : Insert a PUT request in the cache on http://my-api.fr/user/10 whose answer is {"id": 10, "name": "jean"}

In [19]:
# Define the key using HTTP method and URL (without protocol)
request_key = "PUT:/user/10"

# Define the response content (JSON response as a string)
response_content = '{"id": 10, "name": "jean"}'

# Store the request and response content in Redis
r.set(request_key, response_content)

# Optionally, retrieve the cached response to verify
cached_response = r.get(request_key)
print(cached_response.decode('utf-8'))  # Decode bytes to string for printing


{"id": 10, "name": "jean"}


**Q** : Insert a GET request in the cache on http://my-api.fr/user?size=2 whose answer is[{"id": 10, "name": jean}, {"id": 11, "name": "Claire"}]

In [20]:
# Define the key using HTTP method and URL (without protocol)
request_key = "GET:/user?size=2"

# Define the response content (JSON response as a string)
response_content = '[{"id": 10, "name": "jean"}, {"id": 11, "name": "Claire"}]'

# Store the request and response content in Redis
r.set(request_key, response_content)

# Optionally, retrieve the cached response to verify
cached_response = r.get(request_key)
print(cached_response.decode('utf-8'))  # Decode bytes to string for printing


[{"id": 10, "name": "jean"}, {"id": 11, "name": "Claire"}]


**Q** : Create a set of cache request keys.

In [21]:


# Add request keys to the set
r.sadd("cache_keys_set", "GET:/user?size=2")
r.sadd("cache_keys_set", "PUT:/user/10")

# Optionally, retrieve and display the set of request keys
cache_keys = r.smembers("cache_keys_set")
print([key.decode('utf-8') for key in cache_keys])  # Decode each byte to string


['PUT:/user/10', 'GET:/user?size=2']


**Q** : Check if the GET request on http: //http://my-api.fr/user

In [22]:
get_request = 'GET:/user?size=2'
# Check if the GET request on http://my-api.fr/user exists in the cache
exists = r.sismember("cache_keys_set", get_request)
print(f"GET request cache exists: {exists}")

GET request cache exists: 1


**Q** : Delete the PUT request on http://my-api.fr/user/10 from the cache.

In [23]:
# Delete the PUT request on http://my-api.fr/user/10 from the cache
put_request_key = "PUT:/user/10"
r.srem("cache_keys_set", put_request_key)

1

In [24]:
exists = r.sismember("cache_keys_set", put_request_key)
print(f"GET request cache exists: {exists}")

GET request cache exists: 0


## Postquisites

The folloinwg command removes all data from your Redis cluster.

In [25]:
r.flushall()

True